In [13]:
pip install biomart


Note: you may need to restart the kernel to use updated packages.


In [1]:
import gseapy as gp
import scanpy as sc
import numpy as np

In [3]:
# Pfad zur vorhandenen DEG-Liste (ersetzen Sie mit Ihrem Pfad)
deg_file = '/storage/users/sac43cg/res_Samantha_1/outs/DEGs/CTRL_1_vs_CTRL_2D/filtered_DEGs_with_names.csv'
with open(deg_file, 'r') as file:
    content = file.read()

print(content)

0,1,gene_name
ENSG00000164032,2.475180149078369,H2AZ1
ENSG00000197451,2.333972215652466,HNRNPAB
ENSG00000101361,1.9885612726211548,NOP56
ENSG00000203760,2.511373281478882,CENPW
ENSG00000119969,2.848776340484619,HELLS
ENSG00000198830,1.5403218269348145,HMGN2
ENSG00000119403,2.4142990112304688,PHF19
ENSG00000164611,2.5978822708129883,PTTG1
ENSG00000092853,3.389839172363281,CLSPN
ENSG00000108106,2.0547120571136475,UBE2S
ENSG00000130816,2.240041732788086,DNMT1
ENSG00000189159,1.4987751245498655,JPT1
ENSG00000123416,1.7943466901779177,TUBA1B
ENSG00000088325,2.6473031044006348,TPX2
ENSG00000186575,2.411507368087769,NF2
ENSG00000161243,1.598447561264038,FBXO27
ENSG00000103187,1.793815016746521,COTL1
ENSG00000138180,2.5480756759643555,CEP55
ENSG00000120802,2.16248631477356,TMPO
ENSG00000171848,2.584831476211548,RRM2
ENSG00000167641,1.3622283935546875,PPP1R14A
ENSG00000072274,1.9724713563919067,TFRC
ENSG00000166803,1.9554738998413088,PCLAF
ENSG00000198901,2.3299310207366943,PRC1
ENSG00000167900

In [9]:
#Neu
import pandas as pd
from urllib.parse import quote_plus

# Define the Biomart server and dataset name
server = "http://www.ensembl.org/biomart/martservice"
dataset_name = "hsapiens_gene_ensembl"

In [12]:
# Read gene IDs and logfold changes from the text file
gene_data_file = "/storage/users/sac43cg/res_Samantha_1/outs/DEGs/CTRL_1_vs_CTRL_2D/differentially_expressed_genes1.txt"
gene_df = pd.read_csv(gene_data_file, sep="\t", header=None, names=["ensembl_gene_id", "logfold_change"])


print(gene_df)

      ensembl_gene_id  logfold_change
0     ENSG00000164032        2.475180
1     ENSG00000197451        2.333972
2     ENSG00000101361        1.988561
3     ENSG00000203760        2.511373
4     ENSG00000119969        2.848776
...               ...             ...
1123  ENSG00000170345       -5.866974
1124  ENSG00000122884       -2.568223
1125  ENSG00000176171       -2.340556
1126  ENSG00000120738       -4.969254
1127  ENSG00000234741       -2.770039

[1128 rows x 2 columns]


In [13]:
import pandas as pd

# Separate Ensembl gene IDs and logfold changes into separate DataFrames
ensembl_df = gene_df[["ensembl_gene_id"]]
logfold_df = gene_df[["logfold_change"]]

# Display the Ensembl gene IDs DataFrame
print(ensembl_df.head())

# Display the logfold changes DataFrame
print(logfold_df.head())


   ensembl_gene_id
0  ENSG00000164032
1  ENSG00000197451
2  ENSG00000101361
3  ENSG00000203760
4  ENSG00000119969
   logfold_change
0        2.475180
1        2.333972
2        1.988561
3        2.511373
4        2.848776


In [14]:
# Split gene IDs into smaller batches
batch_size = 50
gene_batches = [ensembl_df[i:i+batch_size] for i in range(0, len(ensembl_df), batch_size)]


In [15]:
# Define a list to store Biomart data
biomart_data_list = []


In [16]:
# Iterate through each batch and query Biomart
for batch_df in gene_batches:
    # Create a comma-separated list of gene IDs for the query
    gene_id_list = ",".join(batch_df["ensembl_gene_id"].tolist())

    # Define the query
    query = f"""
        <Query virtualSchemaName="default" formatter="CSV" header="0">
            <Dataset name="{dataset_name}" interface="default">
                <Attribute name="ensembl_gene_id" />
                <Attribute name="external_gene_name" />
            </Dataset>
            <Filter name="ensembl_gene_id" value="{gene_id_list}" />
        </Query>
    """

    # Encode the query
    encoded_query = quote_plus(query)

    # Construct the URL
    url = f"{server}?query={encoded_query}"

    # Read the Biomart data into a DataFrame
    biomart_data_batch = pd.read_csv(url, sep="\t")

    # Append the batch data to the list
    biomart_data_list.append(biomart_data_batch)


In [18]:
print(biomart_data.head())

     ENSG00000210049,MT-TF
0  ENSG00000211459,MT-RNR1
1    ENSG00000210077,MT-TV
2  ENSG00000210082,MT-RNR2
3   ENSG00000209082,MT-TL1
4   ENSG00000198888,MT-ND1


In [19]:
import pandas as pd

# Annahme: Die Geninformationen sind in der ersten Spalte deines DataFrames
gene_names = biomart_data.iloc[:, 0].str.split(',', expand=True)[1]
print(gene_names)

0          MT-RNR1
1            MT-TV
2          MT-RNR2
3           MT-TL1
4           MT-ND1
            ...   
1612640     CD24P1
1612641       DDI2
1612642     RSC1A1
1612643           
1612644    PLEKHM2
Name: 1, Length: 1612645, dtype: object


In [22]:
import pandas as pd

# Read gene IDs and logfold changes from the text file
gene_data_file = "/storage/users/sac43cg/res_Samantha_1/outs/DEGs/CTRL_1_vs_CTRL_2D/differentially_expressed_genes1.txt"
gene_df = pd.read_csv(gene_data_file, sep="\t", header=None, names=["ensembl_gene_id", "logfold_change"])

# Annahme: Die Geninformationen sind in der ersten Spalte deines DataFrames
gene_names = biomart_data.iloc[:, 0].str.split(',', expand=True)[1]

# Create a DataFrame from the gene_names Series
gene_names_df = gene_names.to_frame()

# Merge gene_names_df with logfold_df using their indices
merged_df = gene_names_df.merge(logfold_df, left_index=True, right_index=True)

# Display the merged DataFrame
print(merged_df.head())

# Display the merged DataFrame
print(merged_df.head())

         1  logfold_change
0  MT-RNR1        2.475180
1    MT-TV        2.333972
2  MT-RNR2        1.988561
3   MT-TL1        2.511373
4   MT-ND1        2.848776
         1  logfold_change
0  MT-RNR1        2.475180
1    MT-TV        2.333972
2  MT-RNR2        1.988561
3   MT-TL1        2.511373
4   MT-ND1        2.848776


In [23]:
# Speichere die Gen-Namen in einer Textdatei
merged_df.to_csv('/storage/users/sac43cg/res_Samantha_1/outs/DEGs/CTRL_1_vs_CTRL_2D/differentially_expressed_genes_NEU.txt', index=False, header=False)

In [24]:
# Pfad zur vorhandenen DEG-Liste (ersetzen Sie mit Ihrem Pfad)
deg_file_Neu = '/storage/users/sac43cg/res_Samantha_1/outs/DEGs/CTRL_1_vs_CTRL_2D/differentially_expressed_genes_NEU.txt'

In [25]:
# Path to your GMT file with gene sets
gmt_file = "/storage/users/sac43cg/res_Samantha_1/outs/GMT.files/HALLMARK_GLYCOLYSIS.v2023.1.Hs.gmt"  # Replace with the path to your GMT file


In [26]:

with open(gmt_file, 'r') as gmt_file:
    gmt_content = gmt_file.read()

print(gmt_content)

HALLMARK_GLYCOLYSIS	https://www.gsea-msigdb.org/gsea/msigdb/human/geneset/HALLMARK_GLYCOLYSIS	ABCB6	ADORA2B	AGL	AGRN	AK3	AK4	AKR1A1	ALDH7A1	ALDH9A1	ALDOA	ALDOB	ALG1	ANG	ANGPTL4	ANKZF1	ARPP19	ARTN	AURKA	B3GALT6	B3GAT1	B3GAT3	B3GNT3	B4GALT1	B4GALT2	B4GALT4	B4GALT7	BIK	BPNT1	CACNA1H	CAPN5	CASP6	CD44	CDK1	CENPA	CHPF	CHPF2	CHST1	CHST12	CHST2	CHST4	CHST6	CITED2	CLDN3	CLDN9	CLN6	COG2	COL5A1	COPB2	CTH	CXCR4	CYB5A	DCN	DDIT4	DEPDC1	DLD	DPYSL4	DSC2	ECD	EFNA3	EGFR	EGLN3	ELF3	ENO1	ENO2	ERO1A	EXT1	EXT2	FAM162A	FBP2	FKBP4	FUT8	G6PD	GAL3ST1	GALE	GALK1	GALK2	GAPDHS	GCLC	GFPT1	GFUS	GLCE	GLRX	GMPPA	GMPPB	GNE	GNPDA1	GOT1	GOT2	GPC1	GPC3	GPC4	GPR87	GUSB	GYS1	GYS2	HAX1	HDLBP	HK2	HMMR	HOMER1	HS2ST1	HS6ST2	HSPA5	IDH1	IDUA	IER3	IGFBP3	IL13RA1	IRS2	ISG20	KDELR3	KIF20A	KIF2A	LCT	LDHA	LDHC	LHPP	LHX9	MDH1	MDH2	ME1	ME2	MED24	MERTK	MET	MIF	MIOX	MPI	MXI1	NANP	NASP	NDST3	NDUFV3	NOL3	NSDHL	NT5E	P4HA1	P4HA2	PAM	PAXIP1	PC	PDK3	PFKFB1	PFKP	PGAM1	PGAM2	PGK1	PGLS	PGM2	PHKA2	PKM	PKP2	PLOD1	PLOD2	PMM2	POLR3K	PPFIA4	PPIA	PPP2CB

In [42]:
# Führen Sie GSEA mit gseapy durch
result = gp.prerank(
    rnk='/storage/users/sac43cg/res_Samantha_1/outs/DEGs/CTRL_1_vs_CTRL_2D/differentially_expressed_genes_NEU.txt',  # Pfad zur vorhandenen DEG-Liste
    gene_sets="/storage/users/sac43cg/res_Samantha_1/outs/GMT.files/HALLMARK_GLYCOLYSIS.v2023.1.Hs.gmt",
    processes=16,  # Anzahl der CPU-Prozesse
    permutation_num=1000,  # Anzahl der Permutationen für die p-Wert-Berechnung
    outdir='/storage/users/sac43cg/res_Samantha_1/outs/DEGs/CTRL_1_vs_CTRL_2D/gsea_results'  # Ausgabeordner für Ergebnisse
)

# Zeigen Sie die Ergebnisse an
print(result.res2d)

#Ersetzen Sie 'path/to/deg_list.txt' durch den tatsächlichen Pfad zu Ihrer DEG-Liste und 'path/to/gmt_file.gmt' durch den Pfad zur GMT-Datei mit den Gene-Set-Definitionen.

#Dieses Skript verwendet die vorhandene DEG-Liste, um GSEA mit gseapy durchzuführen und speichert die Ergebnisse im angegebenen Ausgabeordner. Stellen Sie sicher, dass die DEG-Liste und die GMT-Datei korrekt vorhanden und zugänglich sind.


/tmp/ipykernel_438887/3294708760.py:2: DeprecationWarning: processes is deprecated; use threads
  result = gp.prerank(


IndexError: index 1 is out of bounds for axis 0 with size 1

In [41]:
#NOTE: To speed up, use gp.prerank instead with your own ranked list.
import gseapy as gp

# Load the ranked list of DEGs
deg_file_path = '/storage/users/sac43cg/res_Samantha_1/outs/DEGs/CTRL_1_vs_CTRL_2D/differentially_expressed_genes_NEU.txt'
# Load your data, e.g., using pandas, and extract class labels
# deg_data = pd.read_csv(deg_file_path)
# cls_labels = deg_data["class_labels_column_name"]

# Perform GSEA
res = gp.gsea(
    rnk=deg_file_path,
    gene_sets="path/to/gmt_file.gmt",
    cls=cls_labels,  # Provide your class labels here
    permutation_num=1000,
    permutation_type='phenotype',
    outdir=None,
    method='signal_to_noise',
    threads=16
)

AttributeError: 'str' object has no attribute 'obs'

In [48]:
import gseapy as gp
import pandas as pd

# Load your data
deg_data = pd.read_csv("/storage/users/sac43cg/res_Samantha_1/outs/DEGs/CTRL_1_vs_CTRL_2D/differentially_expressed_genes_NEU.txt", header=None)

# Assuming your data is structured as [gene_name, ranking_score] separated by commas
# You can extract gene names and ranking scores
gene_names = deg_data[0]
ranking_scores = deg_data[1].astype(float)

# Create a DataFrame with gene names and ranking scores
ranked_genes = pd.DataFrame({"gene_name": gene_names, "ranking_score": ranking_scores})

# Perform GSEA using prerank
res = gp.prerank(
    rnk=ranked_genes,  # Provide the DataFrame with gene names and ranking scores
    gene_sets="/storage/users/sac43cg/res_Samantha_1/outs/GMT.files/HALLMARK_GLYCOLYSIS.v2023.1.Hs.gmt",
    threads=16,  # Use threads instead of processes
    permutation_num=1000,
    outdir=None
)


2023-08-29 23:31:05,956 [WARNING] Input gene rankings contains NA values(gene name and ranking value), drop them all!
2023-08-29 23:31:05,962 [WARNING] Input gene rankings contains duplicated IDs, Only use the duplicated ID with highest value!
2023-08-29 23:31:05,968 [ERROR] No gene sets passed through filtering condition !!! 
Hint 1: Try to lower min_size or increase max_size !
Hint 2: Check gene symbols are identifiable to your gmt input.
Hint 3: Gene symbols curated in Enrichr web services are all upcases.



KeyError: 'HALLMARK_GLYCOLYSIS'

In [3]:
# Convert gene symbols to Entrez IDs (if necessary)
# entrez_ids = convert_gene_symbols_to_entrez(adata.var_names)
# Extract expression values from AnnData
expression_data = np.array(adata.X)
gene_symbols = adata.var_names

# Create a gene universe (all genes in your dataset)
gene_universe = gene_symbols

In [4]:
# Path to your GMT file with gene sets
gmt_file = "/storage/users/sac43cg/res_Samantha_1/outs/GMT.files/HALLMARK_GLYCOLYSIS.v2023.1.Hs.gmt"  # Replace with the path to your GMT file


In [6]:
# Perform GSEA using gseapy
result = gp.prerank(
    rnk=expression_data,
    gene_sets=gmt_file,  # Replace with the path to your GMT file
    threads=4,  # Number of CPU threads to use
    permutation_num=1000,  # Number of permutations for p-value calculation
    outdir=None  # Output directory for results (None for no output)
)


Exception: Error parsing gene ranking values!

In [ ]:
# Print the results
print(result.res2d)

# Plot the results
gp.plot.gseaplot(rank_metric=result.ranking, term="GLYCOLYSIS")